# PIC-SURE API for PrecisionLink Biobank

The PrecisionLink Biobank is a Boston Children’s Hospital institutional resource that
provides a centralized source of prospectively consented participants. To date, the
Biobank has enrolled over 15,000 patients and families, who have contributed health
data and biological samples for broad research use. Investigators are able to search the
holdings of the Biobank using the [PrecisionLink Biobank Portal](http://biobank.childrens.harvard.edu/).

The PrecisionLink Biobank Portal grants investigators access to genotype,
biospecimen, and clinical data - including clinical notes - from one search tool.
Furthermore, investigators are able to conduct multi-stage queries with any combination
of data criteria.

The Biobank is constantly growing through prospective enrollment of a broadly
consented cohort of patients and families, who can be re-contacted for additional
research opportunities.

## PIC-SURE python API
### What is PIC-SURE?

The Patient Information Commons Standard Unification of Research Elements (PIC-SURE) platform integrates clinical and genomic data from the PrecisionLink Biobank.

Original data exposed through the PIC-SURE API encompasses a large heterogeneity of data organization underneath. PIC-SURE hides this complexity and exposes the different study datasets in a single tabular format. By simplifying the process of data extraction, it allows investigators to focus on downstream analysis and to facilitate reproducible science.

More about PIC-SURE
The API is available in two different programming languages, python and R, enabling investigators to query the databases the same way using either language.

PIC-SURE is a larger project from which the R/python PIC-SURE API is only a brick. Among other things, PIC-SURE also offers a graphical user interface that allows researchers to explore variables across multiple studies, filter patients that match criteria, and create cohorts from this interactive exploration.

The python API is actively developed by the Avillach Lab at Harvard Medical School.

PIC-SURE API GitHub repo:

https://github.com/hms-dbmi/Access-to-Data-using-PIC-SURE-API/tree/master


 -------   

# Getting your own user-specific security token

**Before running this notebook, please be sure to review the "Get your security token" documentation, which exists in the BCH_Precision_Link [README.md file](https://github.com/hms-dbmi/Access-to-Data-using-PIC-SURE-API/tree/master). It explains about how to get a security token, which is mandatory to access the databases.**

# Environment set-up

### Pre-requisites
- python 3.6 or later
- pip python package manager, already available in most systems with a python interpreter installed ([pip installation instructions](https://pip.pypa.io/en/stable/installing/))

### Install Packages

Install the following:
- packages listed in the `requirements.txt` file (listed below, along with version numbers)
- PIC-SURE API components (from Github)
    - PIC-SURE Adapter 
    - PIC-SURE Client

In [ ]:
!cat requirements.txt

In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

In [ ]:
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-client.git

Import all the external dependencies

In [ ]:
import json
from pprint import pprint

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns

import PicSureHpdsLib
import PicSureClient

from python_lib.utils import get_multiIndex_variablesDict, joining_variablesDict_onCol

Set the display parameter for tables and plots

In [ ]:
# Pandas DataFrame display options
pd.set_option("max.rows", 100)

# Matplotlib display parameters
plt.rcParams["figure.figsize"] = (14,8)
font = {'weight' : 'bold',
        'size'   : 12}
plt.rc('font', **font)

## Connecting to a PIC-SURE resource

The following is required to get access to data through the PIC-SURE API: 
- Network URL
- Resource ID
- User-specific security token

If you have not already retrieved your user-specific token, please refer to the "Get your security token" section of the [README.md](https://github.com/hms-dbmi/Access-to-Data-using-PIC-SURE-API/tree/master/NHLBI_BioData_Catalyst#get-your-security-token) file.

***Note: You will not be able to connect to the PrecisionLink PIC-SURE resource if you are not on the Boston Children's Hospital network. Please ensure that you are connected to the BCH network via one of these options:***
- ***TCH or BCH-R wireless options***
- ***the web or full Virtual Private Network (VPN)***

In [ ]:
PICSURE_network_URL = "https://precisionlink-biobank4discovery.childrens.harvard.edu/picsure/"
resource_id = "6aa47730-3288-4c45-bfa1-5a8730666016"
token_file = "token.txt"

In [ ]:
with open(token_file, "r") as f:
    my_token = f.read()

In [ ]:
client = PicSureClient.Client()
connection = client.connect(PICSURE_network_URL, my_token, True)
adapter = PicSureHpdsLib.Adapter(connection)
resource = adapter.useResource(resource_id)

Two objects are created here: a `connection` and a `resource` object.

Since will only be using a single resource, **the `resource` object is actually the only one we will need to proceed with data analysis hereafter**. 

It is connected to the specific data source ID we specified and enables us to query and retrieve data from this database.

## Getting help with the PIC-SURE API

Each object exposed by the PicSureHpds library has a `help()` method. Calling it without parameters will print out  information about functionalities of this object.

In [ ]:
resource.help()

For instance, this output tells us that this `resource` object has 3 methods, and it gives a quick definition of those methods. 

## Using the *variables dictionary*

Once a connection to the desired resource has been established, we first need to understand which variables are available in the database. To this end, we will use the `dictionary` method of the `resource` object.

A `dictionary` instance enables us to retrieve matching records by searching for a specific term, or to retrieve information about all the available variables, using the `find()` method. For instance, looking for variables containing the term `Calcium` in their names is done this way: 

In [ ]:
dictionary = resource.dictionary()
dictionary_search = dictionary.find("Calcium")

Note: Using the `dictionary.find()` function without arguments will return every entry, as shown in the help documentation.
We included the term "Calcium" as we are only interested in entries related to calcium.

Subsequently, objects created by the `dictionary.find` method expose the search results via 4 different methods: `.count()`, `.keys()`, `.entries()`, and `.DataFrame()`.

In [ ]:
pprint({"Count": dictionary_search.count(), 
        "Keys": dictionary_search.keys()[0:5],
        "Entries": dictionary_search.entries()[0:5]})

**The `.DataFrame()` method enables us to get the result of the dictionary search in a pandas DataFrame format. This way, it allows us to:** 


* Use the various information exposed in the dictionary (patient count, variable type ...) as criteria for variable selection.
* Use the row names of the DataFrame to get the actual variable names to be used in the query, as shown below.

Variable names aren't very pratical to use right away for two reasons:
1. Very long
2. Presence of backslashes that prevent from copy-pasting. 

However, retrieving the dictionary search result in the form of a dataframe can help access the variable names.

In [ ]:
dictionary_search.DataFrame().head()

The dictionary currently returned by the API provides information about the variables, such as:
- observationCount: number of entries with non-null value
- categorical: type of the variables, True if strings, False if numerical
- min/max: only provided for numerical variables
- HpdsDataType: 'phenotypes' or 'genotypes'. Currently, the API only expsoses'phenotypes' variables

## Variable dictionary and pandas multiIndex

We can use a simple user-defined function (`get_multiIndex_variablesDict`) to add a little more information to the variable dictionary and to simplify working with variables names. It takes advantage of pandas MultiIndex functionality [see pandas official documentation on this topic](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html).

Although not an official feature of the API, such functionality illustrates how to quickly select groups of related variables.

Printing the multiIndexed variable Dictionary allows to quickly see the tree-like organization of the variable names. Moreover, original and simplified variable names are now stored respectively in the `varName` and `simplified_varName` columns (simplified variable names is simply the last component of the variable name, which is usually the most informative to know what each variable is about).

In [ ]:
multiindex = get_multiIndex_variablesDict(dictionary_search.DataFrame())
multiindex.head()

Below is a simple example to illustrate the simplicity of using a multiIndex dictionary. Let's say we are interested in every variable in the "diseases of the respiratory system" category.

In [ ]:
full_multiindex = get_multiIndex_variablesDict(dictionary.find().DataFrame()) # Get full dataframe of variables
# Find variables that have "diseases of the respiratory system" in level_2
mask_resp_diseases = [type(i) == str and "Diseases of the respiratory system" in i for i in full_multiindex.index.get_level_values(2)] 
# Filter to those variables
resp_diseases_vars = full_multiindex.loc[mask_resp_diseases, :]
resp_diseases_vars.head()

Although pretty simple, it can be easily combined with other filters to quickly select one or more desired groups of variables.

## Querying and retrieving data: Epilepsy and *GRIN2A*

The second cornerstone of the API is the `query` object. It is how we retrieve data from the resource.

First, we need to create a query object.

In [ ]:
my_query = resource.query()

The query object has several methods that enable to build a query.

| Method | Arguments / Input | Output|
|--------|-------------------|-------|
| query.select.add() | variable names (string) or list of strings | all variables included in the list (no record subsetting)|
| query.require.add() | variable names (string) or list of strings | all variables; only records that do not contain null values for input variables |
| query.anyof.add() | variable names (string) or list of strings | all variables; only records that contain at least one non-null value for input variables |
| query.filter.add() | variable name and additional filtering values | input variable; only records that match filter criteria |

All 4 methods can be combined when building a query. The record eventually returned by the query have to meet all the different specified filters.

### Building the query
In the following example, we are going to build a query to return data the counts of patients who:
- had epilepsy and recurrent seizures
- had data available for the *GRIN2A* gene
- had a *GRIN2A* variant of high severity
- had a novel *GRIN2A* variant

#### Add phenotypic variable: Epilepsy

First, let's review the possible variable names with the term "epilepsy".

In [ ]:
epilepsy_search = dictionary.find("epilepsy").DataFrame()
epilepsy_multiindex = get_multiIndex_variablesDict(epilepsy_search)
epilepsy_multiindex.head()

We are interested in all variables pertaining to "Epilepsy and recurrent seizures", which is listed in level_4 of the multiIndex dataframe. We will now filter to all variables that meet this criteria.

In [ ]:
mask_epilepsy = [type(i)==str and "Epilepsy and recurrent seizures" in i for i in epilepsy_multiindex.index.get_level_values(4)]
epilepsy_vars = epilepsy_multiindex.loc[mask_epilepsy, 'name']

Now we can add these variables to our query. We can also view the patient count using the `getCount()` method.

In [ ]:
my_query.anyof().add(epilepsy_vars)
my_query.getCount()

#### Add genomic variable: *GRIN2A*

Next, we will filter to the gene *GRIN2A*. We can do this by filtering the "Gene_with_variant" key to the "GRIN2A" value.

In [ ]:
gene_search = dictionary.find("Gene_with_variant").keys()
gene_search

In [ ]:
my_query.filter().add(gene_search, "GRIN2A")
my_query.getCount()

#### Add genomic variable: Variant severity

We will filter to variants of high severity using the "Variant_severity" key and "High" value.

In [ ]:
severity_search = dictionary.find("Variant_severity").keys()
severity_search

In [ ]:
my_query.filter().add(severity_search, "HIGH")
my_query.getCount()

#### Add genomic variable: Variant frequency

Finally, we will filter to novel variants. Let's take a look at the "Variant_frequency" variable options.

In [ ]:
frequency_search = dictionary.find("Variant_frequency").DataFrame()
frequency_search

As shown above, there are two keys that could be used: "Variant_frequency_as_text" and "Variant_frequency_in_gnomAD". The "Variant_frequency_as_text" has category values of Novel, Rare, and Common. Since we are interested in novel variants, we will use this key moving forward.

In [ ]:
my_query.filter().add("Variant_frequency_as_text", "Novel")
my_query.getCount()

### Retrieving query counts: Genetic associations of different types of epilepsy

Blurb about the article and exploring these relationships

In [ ]:
# Childhood absense epilepsy - GABRG2
# Juvenile myoclonic epilepsy - CACNB4, CLCN2, EFHC1, GABRD
# Febrile Seizures Evaluation - GABRG2, GPR8, SCN1A, SCN9A
# Benign focal epilepsy of childhood - for fun!!

In [ ]:
epi_query = resource.query()

In [ ]:
epilepsy_vars_subset = epilepsy_vars[0:10]
epi_query.crosscounts().add(epilepsy_vars_subset)

In [ ]:
gene_list = ['GABRG2', 'SCN1A', 'CLCN2', 'EFHC1', 'GABRD']#, ['GABRG2', 'SCN1A', 'SCN9A']]

In [ ]:
full_counts = []
results = []
for genes in gene_list:
    epi_query.filter().add("Variant_severity", "HIGH")
    counts = epi_query.getCrossCounts().values()
    for i in counts:
        full_counts.append(i)
    epi_query.filter().delete("Variant_severity")
    epi_query.filter().add("Gene_with_variant", genes)
    res = epi_query.getCrossCounts().values()
    for i in res:
        results.append(i)
    epi_query.filter().delete("Gene_with_variant")

In [ ]:
final = np.array(results).reshape(5, 10)
final = np.transpose(final)
print(final)

full_final = np.array(full_counts).reshape(5, 10)
full_final = np.transpose(full_final)
print(full_final)

full_results = final/full_final
print(full_results)

In [ ]:
len(epilepsy_vars)

In [ ]:
for_plotting = []
for var in epilepsy_vars_subset:
    for_plotting.append(var.split('\\')[-2])
print(for_plotting)

In [ ]:
ax = sns.heatmap(full_results, yticklabels = for_plotting, xticklabels=gene_list)
ax.set_title("Percentage of patients with gene variants and epilepsy")
plt.show()